# Universidade de Fortaleza
## Mestrado em Informática Aplicada
### Ciência de Dados aplicada à Ciência da Cidade

# Final Assignment: Optimal Placement of Government Services in Ceará

## Objective

This notebook applies urban data science concepts to determine optimal placement of government services or private businesses in Ceará municipalities, using socioeconomic metrics from IPEADATA.

## Course Concepts Applied

1. **Voronoi Diagrams** - Service area allocation and coverage analysis
2. **Gravity Models** - Distance decay effects on accessibility

## Data Sources

- **IPEADATA** (http://www.ipeadata.gov.br/) - Socioeconomic data by municipality
- **IBGE** - Geographic coordinates and municipal boundaries
- **Municípios Brasileiros Dataset** - Latitude/longitude for all municipalities

---
## 1. Setup and Imports

In [1]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Geospatial
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.spatial.distance import cdist
from scipy.optimize import minimize

# Optional: for maps
# import folium
# import geopandas as gpd

# IPEADATA API wrapper
import ipeadatapy as ipea

plt.style.use('seaborn-v0_8-whitegrid')

---
## 2. Data Loading

### 2.1 Load Ceará Municipalities with Population and Coordinates

We'll use the existing population data from 02-zipf and add geographic coordinates.

In [2]:
# Load population data from existing zipf analysis
df_pop = pd.read_csv("../02-zipf/ceara_cities.csv", skiprows=1)
df_pop.columns = [c.strip().replace("\ufeff", "") for c in df_pop.columns]

# Rename columns for clarity
df_pop = df_pop.rename(columns={"Código": "code", "Município": "city", "2022": "population"})
df_pop["population"] = pd.to_numeric(df_pop["population"], errors="coerce")
df_pop = df_pop.dropna(subset=["population"])
df_pop = df_pop[df_pop["population"] > 0]

print(f"Loaded {len(df_pop)} municipalities from Ceará")
df_pop.head()

Loaded 184 municipalities from Ceará


,Sigla,code,city,population,Unnamed: 4
0,CE,2300101,Abaiara,10038.0,NaN
1,CE,2300150,Acarapé,14027.0,NaN
2,CE,2300200,Acaraú,65264.0,NaN
3,CE,2300309,Acopiara,44962.0,NaN
4,CE,2300408,Aiuaba,14076.0,NaN


In [3]:
# Load geographic coordinates
# Data source: https://github.com/kelvins/Municipios-Brasileiros
# Download CSV with lat/long for Brazilian municipalities

# Option 1: Download from GitHub
url_coords = "https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/municipios.csv"
df_coords = pd.read_csv(url_coords)

# Filter for Ceará (UF code 23)
df_coords_ce = df_coords[df_coords["codigo_uf"] == 23].copy()
df_coords_ce = df_coords_ce.rename(columns={"codigo_ibge": "code", "nome": "city_name", 
                                             "latitude": "lat", "longitude": "lon"})

print(f"Loaded coordinates for {len(df_coords_ce)} Ceará municipalities")
df_coords_ce.head()

Loaded coordinates for 184 Ceará municipalities


,code,city_name,lat,lon,capital,codigo_uf,siafi_id,ddd,fuso_horario
5,2300101,Abaiara,-7.34588,-39.0416,0,23,1301,88,America/Sao_Paulo
19,2300150,Acarape,-4.22083,-38.7055,0,23,1231,85,America/Sao_Paulo
20,2300200,Acaraú,-2.88769,-40.1183,0,23,1303,88,America/Sao_Paulo
24,2300309,Acopiara,-6.08911,-39.4480,0,23,1305,88,America/Sao_Paulo
83,2300408,Aiuaba,-6.57122,-40.1178,0,23,1307,88,America/Sao_Paulo


In [4]:
# Merge population and coordinates
df = df_pop.merge(df_coords_ce[["code", "lat", "lon"]], on="code", how="inner")
print(f"Merged dataset: {len(df)} municipalities with population and coordinates")
df.head()

Merged dataset: 184 municipalities with population and coordinates


,Sigla,code,city,population,Unnamed: 4,lat,lon
0,CE,2300101,Abaiara,10038.0,NaN,-7.34588,-39.0416
1,CE,2300150,Acarapé,14027.0,NaN,-4.22083,-38.7055
2,CE,2300200,Acaraú,65264.0,NaN,-2.88769,-40.1183
3,CE,2300309,Acopiara,44962.0,NaN,-6.08911,-39.4480
4,CE,2300408,Aiuaba,14076.0,NaN,-6.57122,-40.1178


### 2.2 Load Socioeconomic Data from IPEADATA

We'll fetch employment/unemployment or other relevant metrics.

In [11]:
# List available regional series
series = ipea.list_series()
regional_series = series[series['NAME'].str.contains('emprego', case=False, na=False)]
print(regional_series[['CODE', 'NAME']].head(200))

# For now, we'll use population as the primary metric
# and can add IDHM, PIB per capita, or employment data later

                       CODE                                               NAME
1179        WEO_DESEMWEOBRA                       Taxa de desemprego (FMI/WEO)
1209             DISOC_DESE                       Taxa de desemprego - INATIVA
1210             PAN12_TD12                       Taxa de desemprego - INATIVA
1211            PME12_TDA12  Taxa de desemprego - aberto - referência: 30 d...
1250       PNADC12_TDESOC12                   Taxa de desemprego (desocupação)
1260       SEADE12_TDAGSP12       Taxa de desemprego - aberto - RMSP - INATIVA
2136              Dese_Pnad            Taxa de desemprego (IBGE/Pnad)- INATIVA
2174       PNADCT_TXDSCUPUF            Taxa de desemprego (IBGE/Pnad Contínua)
2267   PNADCT_TXDSCUPUF_BRA  Taxa de desemprego - cor branca (IBGE/Pnad Con...
2268   PNADCT_TXDSCUPUF_NEG  Taxa de desemprego - cor negra (IBGE/Pnad Cont...
2269   PNADCT_TXDSCUPUF_PRD  Taxa de desemprego - cor parda (IBGE/Pnad Cont...
2352   PNADCT_TXDSCUPUF_HOM    Taxa de desemprego - 

---
## 3. Exploratory Analysis

### 3.1 Visualize Municipality Distribution

In [ ]:
# Plot municipalities on a map
fig, ax = plt.subplots(figsize=(10, 10))

# Size points by population (log scale for visibility)
sizes = np.log10(df["population"]) * 20

scatter = ax.scatter(df["lon"], df["lat"], s=sizes, alpha=0.6, 
                     c=df["population"], cmap="YlOrRd", edgecolors="black", linewidth=0.5)

# Highlight Fortaleza
fortaleza = df[df["city"] == "Fortaleza"]
if len(fortaleza) > 0:
    ax.scatter(fortaleza["lon"], fortaleza["lat"], s=300, c="blue", 
               marker="*", label="Fortaleza", zorder=5)

ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("Ceará Municipalities by Population (2022)")
plt.colorbar(scatter, label="Population")
ax.legend()
plt.tight_layout()
plt.show()

---
## 4. Service Placement Problem

### Problem Definition

Given $n$ municipalities with:
- Coordinates $(x_i, y_i)$
- Population $P_i$
- (Optional) Socioeconomic need metric $N_i$

Find optimal locations for $k$ service centers that minimize weighted distance to the population.

### Approaches

1. **Population-weighted centroid** - Single facility location
2. **k-means weighted by population** - Multiple facility location
3. **Gravity model optimization** - Account for distance decay
4. **Voronoi analysis** - Assess current vs optimal coverage

### 4.1 Population-Weighted Centroid

Find the single point that minimizes total weighted distance.

In [ ]:
def weighted_centroid(df):
    """Calculate population-weighted centroid."""
    total_pop = df["population"].sum()
    weighted_lat = (df["lat"] * df["population"]).sum() / total_pop
    weighted_lon = (df["lon"] * df["population"]).sum() / total_pop
    return weighted_lat, weighted_lon

centroid_lat, centroid_lon = weighted_centroid(df)
print(f"Population-weighted centroid: ({centroid_lat:.4f}, {centroid_lon:.4f})")

# Find nearest city to centroid
df["dist_to_centroid"] = np.sqrt((df["lat"] - centroid_lat)**2 + (df["lon"] - centroid_lon)**2)
nearest_city = df.loc[df["dist_to_centroid"].idxmin()]
print(f"Nearest city to centroid: {nearest_city['city']}")

In [ ]:
# Visualize centroid
fig, ax = plt.subplots(figsize=(10, 10))

sizes = np.log10(df["population"]) * 20
ax.scatter(df["lon"], df["lat"], s=sizes, alpha=0.6, c="lightblue", edgecolors="gray")

# Mark centroid
ax.scatter(centroid_lon, centroid_lat, s=200, c="red", marker="X", 
           label="Population-weighted centroid", zorder=5)

# Mark Fortaleza for reference
fortaleza = df[df["city"] == "Fortaleza"]
if len(fortaleza) > 0:
    ax.scatter(fortaleza["lon"], fortaleza["lat"], s=200, c="blue", 
               marker="*", label="Fortaleza", zorder=5)

ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("Optimal Single Facility Location (Population-Weighted Centroid)")
ax.legend()
plt.tight_layout()
plt.show()

### 4.2 Weighted K-Means for Multiple Facilities

Find $k$ optimal locations for service centers.

In [ ]:
def weighted_kmeans(df, k, max_iter=100):
    """
    Population-weighted k-means clustering.
    Each iteration assigns cities to nearest center, then recalculates
    centers as population-weighted centroids of assigned cities.
    """
    np.random.seed(42)
    coords = df[["lat", "lon"]].values
    weights = df["population"].values
    
    # Initialize centers from random cities (weighted by population)
    probs = weights / weights.sum()
    initial_idx = np.random.choice(len(df), size=k, replace=False, p=probs)
    centers = coords[initial_idx].copy()
    
    for iteration in range(max_iter):
        # Assign each city to nearest center
        distances = cdist(coords, centers)
        assignments = np.argmin(distances, axis=1)
        
        # Recalculate centers as weighted centroids
        new_centers = np.zeros_like(centers)
        for c in range(k):
            mask = assignments == c
            if mask.sum() > 0:
                cluster_weights = weights[mask]
                total_weight = cluster_weights.sum()
                new_centers[c] = (coords[mask] * cluster_weights[:, np.newaxis]).sum(axis=0) / total_weight
            else:
                new_centers[c] = centers[c]  # Keep old center if no assignments
        
        # Check convergence
        if np.allclose(centers, new_centers):
            break
        centers = new_centers
    
    return centers, assignments

# Find 5 optimal locations
k = 5
centers, assignments = weighted_kmeans(df, k)
print(f"Found {k} optimal facility locations:")
for i, (lat, lon) in enumerate(centers):
    # Find nearest city
    distances = np.sqrt((df["lat"] - lat)**2 + (df["lon"] - lon)**2)
    nearest = df.loc[distances.idxmin(), "city"]
    pop_served = df[assignments == i]["population"].sum()
    print(f"  Center {i+1}: ({lat:.4f}, {lon:.4f}) - nearest: {nearest}, pop served: {pop_served:,}")

In [ ]:
# Visualize k-means results
fig, ax = plt.subplots(figsize=(12, 10))

colors = plt.cm.tab10(np.linspace(0, 1, k))
sizes = np.log10(df["population"]) * 20

for i in range(k):
    mask = assignments == i
    ax.scatter(df.loc[mask, "lon"], df.loc[mask, "lat"], 
               s=sizes[mask], alpha=0.6, c=[colors[i]], label=f"Cluster {i+1}")

# Mark centers
ax.scatter(centers[:, 1], centers[:, 0], s=300, c="black", marker="X", 
           edgecolors="white", linewidth=2, label="Optimal Centers", zorder=5)

ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title(f"Optimal {k}-Facility Placement (Population-Weighted K-Means)")
ax.legend(loc="lower left")
plt.tight_layout()
plt.show()

### 4.3 Voronoi Diagram Analysis

Analyze service areas using Voronoi tessellation.

In [ ]:
# Create Voronoi diagram from optimal centers
# Note: Voronoi needs at least 3 points and works best with more
from scipy.spatial import Voronoi, voronoi_plot_2d

# Use (lon, lat) for Voronoi (x, y order)
center_points = np.column_stack([centers[:, 1], centers[:, 0]])  # (lon, lat)

# Add boundary points to close the Voronoi regions
lon_min, lon_max = df["lon"].min() - 1, df["lon"].max() + 1
lat_min, lat_max = df["lat"].min() - 1, df["lat"].max() + 1

boundary_points = np.array([
    [lon_min, lat_min], [lon_min, lat_max],
    [lon_max, lat_min], [lon_max, lat_max]
])

all_points = np.vstack([center_points, boundary_points])
vor = Voronoi(all_points)

# Plot
fig, ax = plt.subplots(figsize=(12, 10))

# Plot Voronoi diagram
voronoi_plot_2d(vor, ax=ax, show_vertices=False, line_colors='gray', 
                line_width=1, line_alpha=0.6, point_size=0)

# Plot cities
sizes = np.log10(df["population"]) * 15
ax.scatter(df["lon"], df["lat"], s=sizes, alpha=0.6, c="steelblue", edgecolors="gray")

# Plot centers
ax.scatter(center_points[:, 0], center_points[:, 1], s=200, c="red", 
           marker="X", label="Service Centers", zorder=5)

ax.set_xlim(lon_min + 0.5, lon_max - 0.5)
ax.set_ylim(lat_min + 0.5, lat_max - 0.5)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("Voronoi Service Areas for Optimal Facility Placement")
ax.legend()
plt.tight_layout()
plt.show()

### 4.4 Gravity Model Analysis

Apply distance decay to understand accessibility patterns.

The gravity model states that the "attraction" between a service and a population decreases with distance:

$$A_{ij} = \frac{P_i \cdot S_j}{d_{ij}^\beta}$$

Where:
- $P_i$ = population of city $i$
- $S_j$ = "size" of service at location $j$
- $d_{ij}$ = distance between $i$ and $j$
- $\beta$ = distance decay exponent (typically 1-2)

In [ ]:
def calculate_accessibility(df, service_locations, beta=1.5, d0=0.1):
    """
    Calculate accessibility score for each city based on gravity model.
    Higher score = better access to services.
    """
    coords = df[["lat", "lon"]].values
    accessibility = np.zeros(len(df))
    
    for service_lat, service_lon in service_locations:
        distances = np.sqrt((df["lat"] - service_lat)**2 + (df["lon"] - service_lon)**2)
        # Avoid division by zero
        distances = np.maximum(distances, d0)
        accessibility += 1 / (distances ** beta)
    
    return accessibility

# Calculate accessibility with current optimal centers
df["accessibility"] = calculate_accessibility(df, centers)

# Normalize to 0-100 scale
df["accessibility_norm"] = (df["accessibility"] - df["accessibility"].min()) / \
                           (df["accessibility"].max() - df["accessibility"].min()) * 100

print("Accessibility Statistics:")
print(f"  Min: {df['accessibility_norm'].min():.1f}")
print(f"  Max: {df['accessibility_norm'].max():.1f}")
print(f"  Mean: {df['accessibility_norm'].mean():.1f}")
print(f"  Median: {df['accessibility_norm'].median():.1f}")

# Cities with lowest accessibility
print("\nCities with lowest accessibility:")
print(df.nsmallest(10, "accessibility_norm")[["city", "population", "accessibility_norm"]])

In [ ]:
# Visualize accessibility
fig, ax = plt.subplots(figsize=(12, 10))

sizes = np.log10(df["population"]) * 20
scatter = ax.scatter(df["lon"], df["lat"], s=sizes, c=df["accessibility_norm"], 
                     cmap="RdYlGn", alpha=0.7, edgecolors="gray")

# Mark centers
ax.scatter(centers[:, 1], centers[:, 0], s=200, c="black", marker="X", 
           edgecolors="white", linewidth=2, label="Service Centers", zorder=5)

ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("Accessibility Score by Municipality (Gravity Model)")
plt.colorbar(scatter, label="Accessibility Score (0-100)")
ax.legend()
plt.tight_layout()
plt.show()

---
## 5. Conclusions and Recommendations

### Key Findings

1. **Optimal Locations**: The k-means analysis identified optimal service center locations that minimize population-weighted distances.

2. **Accessibility Gaps**: Cities with lowest accessibility scores represent priority areas for service expansion.

### Recommendations

1. Consider placing new government services near the identified optimal center locations.

2. Prioritize low-accessibility municipalities for service expansion.

---
## References

- IPEADATA: http://www.ipeadata.gov.br/
- IBGE: https://www.ibge.gov.br/
- Municípios Brasileiros Dataset: https://github.com/kelvins/Municipios-Brasileiros
- ipeadatapy documentation: https://www.luanborelli.net/ipeadatapy/docs/